In [1]:
import urllib
from sqlalchemy import create_engine

import pyodbc as pc
import sqlalchemy
import pandas as pd
import numpy as np

import win32com.client

In [94]:
def acad_table_marsrut():
    app = win32com.client.Dispatch("AutoCAD.Application")
    doc = app.ActiveDocument
    model = doc.ModelSpace

    ls = []
    for ob in model:
        ls.append([ob.Handle, ob.EntityName])
    df_ent_type = pd.DataFrame(ls, columns=['handle', 'entity_name'])

    df = df_ent_type[df_ent_type['entity_name']=='AcDbTable']
    b1 = model.Database.HandleToObject(df.iloc[1,0])

    ls = []
    bm = ''
    sm = ''
    for r in range(2, b1.Rows):
        em = b1.GetCellValue(r, 4)
        if em is not None:
            bm1 = b1.GetCellValue(r, 2)
            if bm1 is not None:
                bm = bm1


            if em[:2] == 'До':
                em1 = em[14:]
                em2 = 'До'
            if em[:2] == 'За':
                em1 = em[13:]
                em2 = 'За'
            if em[:2] == 'На':
                em1 = em[9:]
                em2 = 'На'


            sm = b1.GetCellValue(r, 5)
            if sm is None:
                sm = ''

            ls.append([bm,em1,em2,sm])
            # ls.append([bm,em1,em2])

    df1 = pd.DataFrame(ls, columns=['bm', 'em', 'em2', 'sm'])
    # df1 = pd.DataFrame(ls, columns=['bm', 'em', 'em2'])
    return df1

In [2]:
def connect():
    connection_string = "Driver=SQL Server;Server=localhost;Database=fabr;Trusted_Connection=Yes;"
    cnxn = pc.connect(connection_string, autocommit=True)
    return cnxn

def read_sql(cnxn, query):
    df1 = pd.read_sql(query, cnxn)
    return df1

def to_sql(cnxn, table, df):
    cur=cnxn.cursor()
    cols = ','.join(df.columns)
    vals = ','.join('?' * len(df.columns))
    query = f'''insert into {table}({cols}) values ({vals})'''
    cur.fast_executemany = True
    cur.executemany(query, df.values.tolist())

def delete_sql(cnxn, table):
    cnxn.execute(f'''delete {table}''')

def close_conn(cnxn):
    cnxn.close()

In [95]:
df = acad_table_marsrut()

In [5]:
df['tm'] = 1

In [3]:
table = 'marshrut'
table1 = 'devmarsh'
query = f'''select * from {table}'''
cnxn = connect()
# to_sql(cnxn, table, df)

In [9]:
delete_sql(cnxn, table)

In [10]:
to_sql(cnxn, table, df)

In [7]:
close_conn(cnxn)

In [36]:
dfm = pd.read_csv('marsh_fabr.csv')

In [12]:
dfm[dfm['bm']=='М11' and (dfm['em2']=='За' or dfm['em2']=='На')]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [15]:
dfm[(dfm['bm']=='М11') & (dfm['em2']=='За')]

,id,bm,em,em2,tm
11,12,М11,М9,За,1
12,13,М11,М15,За,1
13,14,М11,М17,За,1


In [22]:
bm = 'М11'
query = f"bm=='{bm}' and (em2=='За' or em2=='На') and tm==1"
dfm.query(query)[['bm', 'em', 'em2']]

,bm,em,em2
11,М11,М9,За
12,М11,М15,За
13,М11,М17,За


In [34]:
def marsh_last(dfm, bm):
    query = f"bm=='{bm}' and (em2=='За' or em2=='На') and tm==1"
    return dfm.query(query)['em'].tolist()

def marsh_middle(dfm, bm):
    query = f"bm=='{bm}' and em2=='До' and tm==1"
    return dfm.query(query)['em'].tolist()

def marsh_all_bm(dfm, bm, ls1):
    ls = []

    ls_k = marsh_last(dfm, bm)
    for em in ls_k:
        ls.append(ls1 + [em])
    # return ls

    ls_p = marsh_middle(dfm, bm)
    if len(ls_p) == 0:
       return ls
    for pm in ls_p:
       ls_p_k = marsh_all_bm(dfm, pm, ls1 + [pm])
       if ls_p_k:
            ls.extend(ls_p_k)
    return ls


def marsh_all_em(em, ls_all_marsh):
    ls = []
    for lm in ls_all_marsh:
        if em in lm:
            ind1 = lm.index(em)
            l1 = lm[:ind1+1]
            if l1 not in ls:
                ls.append(l1)
    return ls

def marsh_bm_em(dfm, bm, em):
    ls = marsh_all_bm(dfm, bm, [bm])
    ls1 = marsh_all_em(em, ls)
    return ls1

def zip_marsh(dfm, ls):
    ls3 = []
    ls_id = []
    for ls1 in ls:
        ls2 = []
        ls_id1 = []
        count = len(ls1)
        for i in range(count):
            if i == count-1:
                break
            bm = ls1[i]
            em = ls1[i+1]
            marsh_id = dfm.query(f"bm=='{bm}' and em=='{em}'")['id'].iloc[0]
            ls2.append([bm, em])
            ls_id1.append(marsh_id)
        ls3.append(ls2)
        ls_id.append(ls_id1)
    return ls3, ls_id

def write_marsh(path, ls):
    file1 = open(path, "w", encoding='Windows-1251')
    for ss in ls:
        file1.write(ss +'\n')
    file1.close()

def marsh_sv_bm(dfm, bm):
    ls = marsh_all_bm(dfm, bm, [bm])
    ls7 = []
    for lm in ls:
        for m1 in lm:
            if m1 not in ls7:
                ls7.append(m1)
    write_marsh("marsh_sv_bm.txt", ls7)

def marsh_sv_bm1(dfm, bm):
    ls = marsh_all_bm(dfm, bm, [bm])
    ls7 = []
    for lm in ls:
        for m1 in lm:
            if m1 not in ls7:
                ls7.append(m1)
    return ls7

In [46]:
bm = 'М13'
ls1 = marsh_all_bm(df, bm, [bm])
ls2 = marsh_sv_bm1(df, bm)

In [42]:
bm = 'М11'
ls8 = marsh_all_bm(dfm, bm, [bm])

In [43]:
ls9, ls10 = zip_marsh(dfm, ls8)

In [38]:
bm = 'М11'
ls8 = marsh_sv_bm1(dfm, bm)
ls_str = []
ls_rs = []
for m in ls8:




In [41]:
dfm.query(f"bm == '{ls8[0]}'")

,id,bm,em,em2,tm
11,12,М11,М9,За,1
12,13,М11,М15,За,1
13,14,М11,М17,За,1
14,15,М11,М23,До,1
15,16,М11,М25,До,1
16,17,М11,М105,До,1
17,18,М11,М109,До,1


In [90]:
write_marsh('rrr.txt', ls8)

TypeError: can only concatenate list (not "str") to list

In [59]:
bm = 'М11'
em = 'М63'
ls = marsh_bm_em(dfm, bm, em)
ls = sorted(ls, key=lambda x: len(x))
ls1, ls2 = zip_marsh(dfm, ls)
ls, ls1, ls2

([['М11', 'М23', 'М63'],
  ['М11', 'М23', 'М29', 'М63'],
  ['М11', 'М25', 'М29', 'М63']],
 [[['М11', 'М23'], ['М23', 'М63']],
  [['М11', 'М23'], ['М23', 'М29'], ['М29', 'М63']],
  [['М11', 'М25'], ['М25', 'М29'], ['М29', 'М63']]],
 [[15, 29], [15, 28, 34], [16, 30, 34]])

In [60]:
ls = marsh_all_bm(dfm, bm, [bm])


[['М11', 'М9'],
 ['М11', 'М15'],
 ['М11', 'М17'],
 ['М11', 'М23', 'М29', '1АП'],
 ['М11', 'М23', 'М29', 'М39', 'М45'],
 ['М11', 'М23', 'М29', 'М39', 'М47'],
 ['М11', 'М23', 'М29', 'М39', 'М49'],
 ['М11', 'М23', 'М29', 'М63', '6П'],
 ['М11', 'М23', 'М29', 'М63', '2АП'],
 ['М11', 'М23', 'М29', 'М63', 'М53', 'М57', '7П'],
 ['М11', 'М23', 'М29', 'М63', 'М53', 'М57', 'М59'],
 ['М11', 'М23', 'М29', 'М100', 'М54', 'М44', 'М28', 'М20', 'М4', 'М8'],
 ['М11', 'М23', 'М29', 'М100', 'М54', 'М44', 'М28', 'М20', 'М4', 'М68', 'М2'],
 ['М11', 'М23', 'М29', 'М100', 'М54', 'М44', 'М28', 'М20', 'М4', 'М68', 'М6'],
 ['М11', 'М23', 'М29', 'М100', 'М54', 'М44', 'М28', 'М20', 'М12', 'М84'],
 ['М11', 'М23', 'М29', 'М100', 'М54', 'М44', 'М28', 'М20', 'М12', 'М138'],
 ['М11', 'М23', 'М29', 'М100', 'М54', 'М44', 'М28', 'М20', 'М12', 'М156'],
 ['М11',
  'М23',
  'М29',
  'М100',
  'М54',
  'М44',
  'М28',
  'М20',
  'М12',
  'М124',
  'АП'],
 ['М11',
  'М23',
  'М29',
  'М100',
  'М54',
  'М44',
  'М28',
  'М20',

In [85]:
def write_marsh(path, ls):
    file1 = open(path, "w", encoding='Windows-1251')
    for ss in ls:
        file1.write(ss +'\n')
    file1.close()

def marsh_sv_bm(dfm, bm):
    ls = marsh_all_bm(dfm, bm, [bm])
    ls7 = []
    for lm in ls:
        for m1 in lm:
            if m1 not in ls7:
                ls7.append(m1)
    write_marsh("marsh_sv_bm.txt", ls7)


In [86]:
marsh_sv_bm(dfm, bm)

In [68]:
file1 = open("MyFile.txt", "w", encoding='Windows-1251')
for ss in ls:
    file1.write(','.join(ss)+'\n')
file1.close()

In [27]:
ls = []
m1 = 'М11'
query = f'''select bm, em, em2 from marshrut
            where bm = '{m1}' and (em2 = 'За' or em2 = 'На') and tm = 1
'''
df1 = read_sql(cnxn, query)
ls = df1[['bm', 'em']].values.tolist()
ls

C:\Users\Alex\AppData\Local\Temp\ipykernel_1716\2466642199.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql(query, cnxn)


[]

In [29]:
query = f'''select bm, em, em2 from marshrut
            where bm = '{m1}' and em2 = 'До' and tm = 1
'''
df1 = read_sql(cnxn, query)
ls1 = df1['em'].tolist()
ls1


C:\Users\Alex\AppData\Local\Temp\ipykernel_1716\2466642199.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql(query, cnxn)


['М23', 'М25', 'М105', 'М109']

In [4]:
def f_k(bm):
    query = f'''select bm, em, em2 from marshrut
                where bm = '{bm}' and (em2 = 'За' or em2 = 'На') and tm = 1
    '''
    df1 = read_sql(cnxn, query)
    ls = df1['em'].tolist()
    return ls

def f_p(bm):
    query = f'''select bm, em, em2 from marshrut
                where bm = '{bm}' and em2 = 'До' and tm = 1
    '''
    df1 = read_sql(cnxn, query)
    ls1 = df1['em'].tolist()
    return ls1

def f_r(bm, ls1):
    ls = []

    ls_k = f_k(bm)
    for em in ls_k:
        ls.append(ls1 + [em])
    # return ls

    ls_p = f_p(bm)
    if len(ls_p) == 0:
       return ls
    for pm in ls_p:
       ls_p_k = f_r(pm, ls1 + [pm])
       if ls_p_k:
            ls.extend(ls_p_k)
    return ls

def marsh1(em, ls_all_marsh):
    ls = []
    for lm in ls_all_marsh:
        if em in lm:
            ind1 = lm.index(em)
            l1 = lm[:ind1+1]
            if l1 not in ls:
                ls.append(l1)
    return ls

def marsh(bm, em):
    ls = f_r(bm, [bm])
    ls1 = marsh1(em, ls)
    return ls1

In [76]:
bm = 'М11'
em = 'М63'
ls = marsh(bm, em)
ls1 = sorted(ls, key=lambda x: len(x))
ls1

C:\Users\Alex\AppData\Local\Temp\ipykernel_1716\2466642199.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql(query, cnxn)
C:\Users\Alex\AppData\Local\Temp\ipykernel_1716\2466642199.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql(query, cnxn)
C:\Users\Alex\AppData\Local\Temp\ipykernel_1716\2466642199.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql(query, cnxn)
C:\Users\Alex\AppData\Local\Temp\ipykernel_1716\2466642199.py:7: UserWarning: pandas only supports SQLA

[['М11', 'М23', 'М63'],
 ['М11', 'М23', 'М29', 'М63'],
 ['М11', 'М25', 'М29', 'М63']]

In [77]:
ls2 = ls1[0]
ls2

['М11', 'М23', 'М63']

In [84]:
def zip_marsh(ls):
    ls3 = []
    count = len(ls)
    for i in range(count):
        if i == count-1:
            break
        ls3.append([ls[i], ls[i+1]])
    return ls3

In [86]:
ls3 = zip_marsh(ls2)
ls3

[['М11', 'М23'], ['М23', 'М63']]

In [87]:
ls4 = ls3[0]
ls4

['М11', 'М23']

In [93]:

for ls4 in ls3:
    query = f'''select id from {table} where bm = '{ls4[0]}' and em = '{ls4[1]}' '''
    df1 = read_sql(cnxn, query)
    id1 = df1.iloc[0,0]
    id1

C:\Users\Alex\AppData\Local\Temp\ipykernel_1716\2466642199.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql(query, cnxn)


27

In [100]:
df.to_excel('marsh_fabr.xlsx', index=False)

In [50]:
def write_marsh():
    df = pd.read_excel('c:/tt/marsh_fabr.xlsx')
    df['id'] = list(range(1, len(df)+1))
    df = df[['id', 'bm', 'em', 'em2', 'sm', 'rs']]
    df['sm'] = df['sm'].str.replace(' ', '')
    df['rs'] = df['rs'].str.replace(' ', '')
    df['sm'].fillna('', inplace=True)
    df['tm'] = 1
    df[['id', 'bm', 'em', 'em2', 'tm']].to_csv('c:/tt/marsh_fabr.csv', index_label=False)
    return df

In [51]:
write_marsh()

,id,bm,em,em2,sm,rs,tm
0,1,Мн,М1,До,,"Т,1ГП",1
1,2,Мз,М7,До,,"Т,2ГП",1
2,3,М1,М203,До,"-1/3,-5","1ГП,1,3-5,5/201П",1
3,4,М1,М11,До,"-1/3,+5","1ГП,1,3-5,5/9П",1
4,5,М1,М13,До,+1/3,"1ГП,1,1/15П",1
...,...,...,...,...,...,...,...
289,290,М216,М218,За,"-208,+204/206,-212","203П,204-212",1
290,291,М216,М206,До,"-208,+204/206,+212","203П,204-212,202/212П",1
291,292,М218,М212,На,"-212,-204/206","Т,204-212,206,201П",1
292,293,М218,М214,На,"-212,+204/206,+208","Т,204-212,202П",1


In [47]:
df1 = pd.read_csv('marsh_fabr.csv')
df1

,id,bm,em,em2,tm
0,1,Мн,М1,До,1
1,2,Мз,М7,До,1
2,3,М1,М203,До,1
3,4,М1,М11,До,1
4,5,М1,М13,До,1
...,...,...,...,...,...
289,290,М216,М218,За,1
290,291,М216,М206,До,1
291,292,М218,М212,На,1
292,293,М218,М214,На,1


In [2]:
df = pd.read_excel('marsh_fabr.xlsx')
df['tm'] = 1
df

,bm,em,em2,sm,rs,tm
0,Мн,М1,До,NaN,"Т, 1ГП",1
1,Мз,М7,До,NaN,"Т, 2ГП",1
2,М1,М203,До,"-1/3, -5","1ГП, 1, 3-5, 5/201П",1
3,М1,М11,До,"-1/3, +5","1ГП, 1, 3-5, 5/9П",1
4,М1,М13,До,+1/3,"1ГП, 1, 1/15П",1
...,...,...,...,...,...,...
289,М216,М218,За,"-208,+204/206,-212","203П,204-212",1
290,М216,М206,До,"-208,+204/206,+212","203П,204-212,202/212П",1
291,М218,М212,На,"-212,-204/206","Т,204-212,206,201П",1
292,М218,М214,На,"-212,+204/206, +208","Т,204-212,202П",1


In [12]:
import numpy as np

In [30]:
def write_str_name():
    df = pd.read_excel('c:/tt/marsh_fabr.xlsx')
    ls = df['sm'].tolist()
    ls_str = []
    for l_st in ls:
        if l_st is not np.nan:
            for st in l_st.split(','):
                st1 = st.strip()
                st1 = st1[1:]
                if st1 not in ls_str:
                    ls_str.append(st1)
    write_to_txt('c:/tt/str_name.txt', ls_str)
    df_str = pd.DataFrame(ls_str, columns=['name'])
    df_str.to_csv('str_name.csv', index_label=False)

def write_rs_name():
    df = pd.read_excel('c:/tt/marsh_fabr.xlsx')
    ls = df['rs'].tolist()
    ls_str = []
    for l_st in ls:
        if l_st is not np.nan:
            for st in l_st.split(','):
                st1 = st.strip()
                if st1 not in ls_str:
                    ls_str.append(st1)
    write_to_txt('c:/tt/rs_name.txt', ls_str)
    df_str = pd.DataFrame(ls_str, columns=['name'])
    df_str.to_csv('rs_name.csv', index_label=False)

def write_to_txt(path, ls):
    with open(path, "w", encoding='Windows-1251') as f:
        for ss in ls:
            f.write(ss +'\n')


In [31]:
write_str_name()
write_rs_name()

In [ ]:
open

In [5]:
df = pd.read_excel('marsh_fabr.xlsx')
df

,bm,em,em2,sm,rs
0,Мн,М1,До,NaN,"Т, 1ГП"
1,Мз,М7,До,NaN,"Т, 2ГП"
2,М1,М203,До,"-1/3, -5","1ГП, 1, 3-5, 5/201П"
3,М1,М11,До,"-1/3, +5","1ГП, 1, 3-5, 5/9П"
4,М1,М13,До,+1/3,"1ГП, 1, 1/15П"
...,...,...,...,...,...
289,М216,М218,За,"-208,+204/206,-212","203П,204-212"
290,М216,М206,До,"-208,+204/206,+212","203П,204-212,202/212П"
291,М218,М212,На,"-212,-204/206","Т,204-212,206,201П"
292,М218,М214,На,"-212,+204/206, +208","Т,204-212,202П"


In [8]:
to_sql(cnxn, table, df[['bm', 'em', 'em2', 'tm']])

In [93]:
df['sm'] = df['sm'].str.replace(' ','')
df['rs'] = df['rs'].str.replace(' ','')

In [94]:
df['sm'].fillna('', inplace=True)

In [100]:
df

,bm,em,em2,sm,rs,tm
0,Мн,М1,До,,"Т,1ГП",1
1,Мз,М7,До,,"Т,2ГП",1
2,М1,М203,До,"-1/3,-5","1ГП,1,3-5,5/201П",1
3,М1,М11,До,"-1/3,+5","1ГП,1,3-5,5/9П",1
4,М1,М13,До,+1/3,"1ГП,1,1/15П",1
...,...,...,...,...,...,...
289,М216,М218,За,"-208,+204/206,-212","203П,204-212",1
290,М216,М206,До,"-208,+204/206,+212","203П,204-212,202/212П",1
291,М218,201П,На,"-212,-204/206","Т,204-212,206,201П",1
292,М218,202П,На,"-212,+204/206,+208","Т,204-212,202П",1


In [ ]:
df.to_csv('marsh_fabr.csv', index_label=False)

In [38]:
ls = []
for index, row in df.iterrows():
    bm = row['bm']
    id_m = row['id']
    sm = row['sm']
    rs = row['rs']
    ls_str = sm.split(',')
    ls_rs = rs.split(',')
    ss = [[id_m, ls_rs[0], 'РЦ', 1, 0, 1], [id_m, bm, 'Светофор', 1, 0, 2]]
    i = 3
    for rs1 in ls_rs[1:]:
        ss.append([id_m, rs1, 'РЦ', 0, 1, i])
        i += 1
    for str1 in ls_str:
        if str1 != '':
            str2 = str1[1:]
            str3 = str1[:1]
            str_pos = 1
            if str3 == '-':
                str_pos = 0
            ss.append([id_m, str2, 'Стрелка', str_pos, 0, 0])
    ls.extend(ss)


In [39]:
df2 = pd.DataFrame(ls, columns=['marshrut_id', 'elm', 'telm', 'ppos', 'isp', 'pnisp'])
df2

,marshrut_id,elm,telm,ppos,isp,pnisp
0,1,Т,РЦ,1,0,1
1,1,Мн,Светофор,1,0,2
2,1,1ГП,РЦ,0,1,3
3,2,Т,РЦ,1,0,1
4,2,Мз,Светофор,1,0,2
...,...,...,...,...,...,...
2159,294,204-212,РЦ,0,1,3
2160,294,203П,РЦ,0,1,4
2161,294,212,Стрелка,0,0,0
2162,294,204/206,Стрелка,1,0,0


In [42]:
to_sql(cnxn, 'devmarsh', df2)

In [4]:
query = f'''select * from {table}

'''
df3 = read_sql(cnxn, query)
df3

C:\Users\Alex\AppData\Local\Temp\ipykernel_3952\2466642199.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql(query, cnxn)


,id,bm,em,em2,tm
0,1,Мн,М1,До,1
1,2,Мз,М7,До,1
2,3,М1,М203,До,1
3,4,М1,М11,До,1
4,5,М1,М13,До,1
...,...,...,...,...,...
289,290,М216,М218,За,1
290,291,М216,М206,До,1
291,292,М218,201П,На,1
292,293,М218,202П,На,1


In [8]:
df3.to_csv('marsh_fabr.csv', index_label=False)

In [87]:
df4 = pd.read_csv('marsh_fabr.csv')
df4

,id,bm,em,em2,tm
0,1,Мн,М1,До,1
1,2,Мз,М7,До,1
2,3,М1,М203,До,1
3,4,М1,М11,До,1
4,5,М1,М13,До,1
...,...,...,...,...,...
289,290,М216,М218,За,1
290,291,М216,М206,До,1
291,292,М218,201П,На,1
292,293,М218,202П,На,1


In [98]:
df4.columns.values.tolist()

['id', 'bm', 'em', 'em2', 'tm']

In [88]:
df4.to_csv('marsh_fabr.csv', index_label=False)

In [101]:
df['id'] = df4['id']
df

,bm,em,em2,sm,rs,tm,id
0,Мн,М1,До,,"Т,1ГП",1,1
1,Мз,М7,До,,"Т,2ГП",1,2
2,М1,М203,До,"-1/3,-5","1ГП,1,3-5,5/201П",1,3
3,М1,М11,До,"-1/3,+5","1ГП,1,3-5,5/9П",1,4
4,М1,М13,До,+1/3,"1ГП,1,1/15П",1,5
...,...,...,...,...,...,...,...
289,М216,М218,За,"-208,+204/206,-212","203П,204-212",1,290
290,М216,М206,До,"-208,+204/206,+212","203П,204-212,202/212П",1,291
291,М218,201П,На,"-212,-204/206","Т,204-212,206,201П",1,292
292,М218,202П,На,"-212,+204/206,+208","Т,204-212,202П",1,293


In [103]:
df[['id', 'bm', 'em', 'em2', 'tm']].to_csv('marsh_fabr.csv', index_label=False)

In [70]:
ls_sv = dfm['bm'].unique().tolist()
ls_sv

['Мн',
 'Мз',
 'М1',
 'М3',
 'М5',
 'М7',
 'М9',
 'М11',
 'М13',
 'М15',
 'М17',
 'М19',
 'М21',
 'М23',
 'М25',
 'М27',
 'М29',
 'М33',
 'М35',
 'М37',
 'М39',
 'М41',
 'М43',
 'М45',
 'М47',
 'М49',
 'М51',
 'М53',
 'М55',
 'М57',
 'М59',
 'М61',
 'М63',
 'М65',
 'М67',
 'М69',
 'М97',
 'М101',
 'М103',
 'М105',
 'М107',
 'М109',
 'Ч101',
 'Ч102',
 'Ч103',
 'Ч104',
 'Ч105',
 'Ч106',
 'Ч107',
 'Ч108',
 'Мк',
 'М2',
 'М4',
 'М6',
 'М8',
 'М10',
 'М12',
 'М14',
 'М16',
 'М18',
 'М20',
 'М22',
 'М26',
 'М28',
 'М30',
 'М32',
 'М34',
 'М36',
 'М38',
 'М40',
 'М42',
 'М44',
 'М46',
 'М48',
 'М50',
 'М52',
 'М54',
 'М56',
 'М58',
 'М60',
 'М62',
 'М64',
 'М66',
 'М68',
 'М70',
 'М72',
 'М74',
 'М80',
 'М82',
 'М84',
 'М100',
 'М102',
 'М104',
 'М106',
 'М108',
 'М110',
 'М112',
 'М114',
 'М116',
 'М118',
 'М120',
 'М122',
 'М124',
 'М126',
 'М128',
 'М130',
 'М132',
 'М134',
 'М136',
 'М138',
 'М140',
 'М142',
 'М144',
 'М146',
 'М148',
 'М150',
 'М152',
 'М154',
 'М156',
 'М158',
 'М160',


In [71]:
file1 = open("sv_name.txt", "w", encoding='Windows-1251')
for ss in ls_sv:
    file1.write(ss +'\n')
file1.close()

In [60]:
l1 = ['М11', 'М23', 'М63', 'М74', 'М66', 'М32', 'М26', 'М20', 'М12', 'М138']

In [61]:
l1.index(em)

2

In [62]:
l1[:2]

['М11', 'М23']